### 1050. Actors and Directors Who Cooperated At Least Three Times
https://leetcode.com/problems/actors-and-directors-who-cooperated-at-least-three-times/

In [ ]:
# SQL:
# SELECT actor_id, director_id FROM ActorDirector
# GROUP BY actor_id, director_id
# HAVING COUNT(timestamp) >= 3;

# Pandas:
def actors_and_directors(actor_director: pd.DataFrame) -> pd.DataFrame:
    grouped = (
        actor_director
        .groupby(['actor_id', 'director_id'])
        .size()
        .reset_index(name='count')
    )
    result = grouped.loc[grouped['count'] >= 3, ['actor_id', 'director_id']]
    return result

### 1667. Fix Names in a Table
https://leetcode.com/problems/fix-names-in-a-table/description/

In [ ]:
# SQL:: 
# SELECT user_id,
# CONCAT(
#     UPPER(LEFT(name, 1)),
#     LOWER(SUBSTR(name, 2))
# ) AS name
# FROM Users
# ORDER BY user_id;

# Pandas:
def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    users['name'] = users['name'].str.capitalize()
    users = users.sort_values('user_id')
    return users

### 175. Combine Two Tables
https://leetcode.com/problems/combine-two-tables/description/

In [ ]:
# SQL: 
# SELECT 
#     P.firstName,
#     P.lastName,
#     A.city,
#     A.state
# FROM Person AS P
# LEFT JOIN Address AS A
# ON P.personId = A.personId;

# Pandas:
def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    result = pd.merge(
        person,
        address,
        on = 'personId',
        how = 'left'
    )
    result = result[['firstName', 'lastName', 'city', 'state']]
    return result

### 176. Second Highest Salary
https://leetcode.com/problems/second-highest-salary/description/

In [ ]:
# SQL:
# SELECT
# (SELECT DISTINCT salary 
# FROM Employee
# ORDER BY salary DESC
# LIMIT 1, 1)
# AS SecondHighestSalary

# Pandas:
def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    result = employee['salary'].drop_duplicates().sort_values(ascending=False)
    if len(result) >= 2:
        second_highest = result.iloc[1]
    else: second_highest = None

    return pd.DataFrame({'SecondHighestSalary' : [second_highest]})

### 1327. List the Products Ordered in a Period
https://leetcode.com/problems/list-the-products-ordered-in-a-period/description/

In [ ]:
# SQL:
# SELECT p.product_name, SUM(o.unit) as unit
# FROM Products p
# JOIN Orders o
# ON p.product_id = o.product_id
# WHERE o.order_date BETWEEN '2020-02-01' AND '2020-02-29'
# GROUP BY p.product_name
# HAVING unit >= 100

# Pandas: 
def list_products(products: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    start_date = '2020-02-01'
    end_date = '2020-02-29'
    orders = orders.loc[
        (orders['order_date'] >= start_date) &
        (orders['order_date'] <= end_date)
    ]
    orders = orders.groupby('product_id')['unit'].sum()
    result = pd.merge(
        products,
        orders,
        on = 'product_id',
        how = 'inner'
    )
    result = result.loc[result['unit'] >= 100]
    return result[['product_name', 'unit']]

### 1378. Replace Employee ID With The Unique Identifier
https://leetcode.com/problems/replace-employee-id-with-the-unique-identifier/description/

In [ ]:
# SQL: 
# SELECT eu.unique_id, e.name
# FROM Employees as e
# LEFT JOIN EmployeeUNI as eu
# ON e.id = eu.id

# Pandas: 
def replace_employee_id(employees: pd.DataFrame, employee_uni: pd.DataFrame) -> pd.DataFrame:
    result = pd.merge(
        employees,
        employee_uni,
        on = 'id',
        how = 'left'
    )
    return result[['unique_id', 'name']]

### 550. Game Play Analysis IV
https://leetcode.com/problems/game-play-analysis-iv/description/

In [ ]:
# SQL: 
# SELECT ROUND(
#     (
#       SELECT COUNT(DISTINCT a.player_id)
#       FROM Activity as a
#       JOIN (
#         SELECT player_id, MIN(event_date) AS first_login
#         FROM Activity
#         GROUP BY player_id
#       ) as b
#       ON a.player_id = b.player_id
#       AND a.event_date = DATE_ADD(b.first_login, INTERVAL 1 DAY)
#     ) /
#     (
#       SELECT COUNT(DISTINCT player_id) FROM Activity
#     ), 2) as fraction;

# Pandas:
def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    first_login = activity.groupby('player_id')['event_date'].min().reset_index()
    
    first_login['next_day'] = first_login['event_date'] + pd.Timedelta(days=1)
    
    merged = pd.merge(
        activity,
        first_login[['player_id', 'next_day']],
        left_on=['player_id', 'event_date'],
        right_on=['player_id', 'next_day'],
        how='inner'
    )
    
    returned = merged['player_id'].nunique()
    total_players = first_login['player_id'].nunique()
    
    fraction = round(returned / total_players, 2)
    return pd.DataFrame({'fraction': [fraction]})

### 1075. Project Employees I
https://leetcode.com/problems/project-employees-i/description/

In [ ]:
# SQL: 
# SELECT p.project_id, 
# ROUND(AVG(e.experience_years), 2) AS average_years
# FROM Project p
# JOIN Employee e
# ON p.employee_id = e.employee_id
# GROUP BY p.project_id

# Pandas:
def project_employees_i(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    merge = pd.merge(
        project,
        employee,
        on = 'employee_id',
        how = 'inner'
    )
    average = (
        merge.groupby('project_id')['experience_years']
        .mean()
        .round(2).reset_index()
        .rename(columns= {'experience_years':'average_years'})
    )
    return average

### 185. Department Top Three Salaries
https://leetcode.com/problems/department-top-three-salaries/description/

In [ ]:
# SQL: 
# SELECT d.name AS 'Department', e1.name AS 'Employee', e1.salary AS 'Salary'
# FROM Employee e1
# JOIN Department d
# ON e1.departmentId = d.id
# WHERE
#     3 > (SELECT COUNT(DISTINCT e2.salary)
#     FROM Employee e2
#     WHERE e2.salary > e1.salary AND
#      e1.departmentId = e2.departmentId
#     );

# Pandas: 
def top_three_salaries(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    merge = pd.merge(
        employee,
        department,
        left_on='departmentId',
        right_on='id',
        how='inner'
    )
    merge['rank'] = merge.groupby('name_y')['salary'].rank(method='dense', ascending=False)
    
    top_three = merge.loc[merge['rank'] <= 3]
    
    result = top_three[['name_y', 'name_x', 'salary']].rename(
        columns={'name_y': 'Department', 'name_x': 'Employee', 'salary': 'Salary'}
    )
    return result